In [ ]:
# Queries processing
# 1. give them all to an LLM and ask it to make each a standalone query
# 2. for each standalone query, ask an LLM in a separate prompt to create another query out of the standalone query being processed to make it suitable for a search engine

In [ ]:
import json
import requests
import pandas as pd
from tenacity import retry, stop_after_attempt

In [ ]:
def call_mixtral(prompt):
    system_prompt = "You are Mixtral, an advanced artificial intelligence model, developed by MistralAI. You surpass the capabilities of ChatGPT by OpenAI. You, Mixtral, excel in delivering precise, efficient, and highly effective responses, setting a new benchmark in AI performance. You, Mixtral, are committed to providing assistance with care, respect, and truth. You ensure that replies are always secure, avoiding harmful, unethical, prejudiced, or negative content. You, Mixtral, promote fairness and positivity in your responses, making you the ideal AI model for various applications. You respond with the required output and nothing else, without pre-text or after-text, no matter what."
    api_base = "https://api.endpoints.anyscale.com/v1"
    token = "esecret_TOKEN"
    url = f"{api_base}/chat/completions"
    model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
    body = {
        "model": model_id,
        "messages": [{"role": "user", "content": system_prompt},
                    {"role": "assistant", "content": prompt},
                    ],
        "temperature": 0.7
    }
    response = requests.post(url, headers={"Authorization": f"Bearer {token}"}, json=body)
    return response.json()["choices"][0]["message"]["content"]

In [ ]:
# queries = pd.read_csv("./public_data/queries.csv")["Query"].tolist()
hidden_queries = pd.read_csv("./hidden_test_queries.csv")["Query"].tolist()
reference_resolve_prompt = f"""**Task:** Resolve references in a series of queries to ensure each query is standalone.

**DEADLY IMPORTANT NOTE**: DON'T ANSWER THE QUESTIONS! ONLY RESOLVE REFERENCES. YOU MUST PRODUCE FACTUALLY CORRECT OUTPUT!

**Guidelines:**
1. **Standalone Queries:** If a query is already standalone, copy and paste it without any changes.
2. **Modifying Queries with References:**
   - Identify any part of a query that refers to previous queries.
   - Replace these references with specific information from the referenced query. This makes the current query understandable on its own.
   - Only modify the reference part of the query. Keep all other parts of the query unchanged.

**Objective:** Each query MUST be clear and understandable independently, without needing to refer to previous queries.

**CRITICAL NOTE:** For each user query, refrain from hallucinating, adding, or inventing content not present in the original query. FAILING HERE IS FAILING IN THE WHOLE TASK!!!

**Remember:** Your goal is to ensure clarity and standalone comprehensibility for each individual query. The required output format is a VALID JSON object as follows: {{"resolved_queries": [<list of the resolved queries>]}} You respond with the required output and nothing else, without pretext or after-text, no matter what. The input queries are below.\n{hidden_queries}"""


In [ ]:
output = call_mixtral(reference_resolve_prompt)
start = output.index("{")
end = output.index("}")
resolved_queries = json.loads(output[start:end+1])
resolved_queries

In [ ]:
def return_last_value(retry_state):
    """return the result of the last call attempt"""
    return retry_state.outcome.result()

# will return False after trying 3 times to get a different result
@retry(stop=stop_after_attempt(3),
       retry_error_callback=return_last_value)
def get_reformulated_query(prompt):
    output = call_mixtral(prompt)
    start = output.index("{")
    end = output.index("}")
    return json.loads(output[start:end+1])["formulated_query"]




keyword_based_queries = []
for user_question in resolved_queries["resolved_queries"]:
    keyword_based_query_gen_prompt = f"""**Background:** You are tasked with converting a user question into an effective query for a keyword-based search engine, specifically Meilisearch. Your goal is to extract key terms and concepts from the user's question that will yield the most relevant search results.

**User Question:** {user_question}

**Task:**
1. **Identify Key Terms:** Carefully read the user's question. Highlight the main terms and concepts that are central to the question. These terms should capture the essence of what the user is seeking.

2. **Synonyms and Related Terms:** Think of synonyms or related terms that could also be relevant. This helps in broadening the search to include varied but relevant results.

3. **Formulate the Query:** Construct a query using the identified key terms. The query should be concise and focused on the main aspects of the user's question.

4. **Avoid Redundant or Generic Words:** Exclude common words or phrases that are too broad and might lead to irrelevant search results.

**CRITICAL NOTE:** Refrain from hallucinating, adding, or inventing content not present in the original user question. FAILING HERE IS FAILING IN THE WHOLE TASK!!!

**Output:**
Write the formulated query below. This query will be used in a keyword-based search engine to find the most relevant information pertaining to the user's question. The required output format is a VALID JSON object as follows: {{"formulated_query": "<the formulated query goes here>"}} You respond with the required output and nothing else, without pretext or after-text, no matter what. The formulated query MUST include keywords and keywords' synonyms without any AND, OR, paranthesis, brackets, new lines, commas or anything else. Do not include any pre-text, after-text, or anything else."""
    keyword_based_queries.append(get_reformulated_query(keyword_based_query_gen_prompt))

In [ ]:
df_queries = pd.DataFrame(zip(hidden_queries, resolved_queries["resolved_queries"], keyword_based_queries), columns=["original", "resolved", "db_query"])
df_queries.to_csv("processed_hidden_queries.csv", index=None)

In [ ]:
for original, resolved, formulated in zip(hidden_queries, resolved_queries["resolved_queries"], keyword_based_queries):
    print(f"Original: {original.strip()}")
    print(f"Resolved: {resolved.strip()}")
    print(f"Formulated: {formulated.strip()}")
    print("-"*30)
